0.3.10


In [8]:
!pip install keras_ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 8.5 MB/s eta 0:00:00


In [9]:
import cv2
import matplotlib.pyplot as plt
import keras_ocr
import math
import numpy as np





# using OCR

In [13]:
import os
import math
import numpy as np
import cv2
import keras_ocr

def midpoint(x1, y1, x2, y2):
    return int((x1 + x2) / 2), int((y1 + y2) / 2)

def inpaint_text(input_folder_path, output_folder_path='output_folder', max_text_regions=4):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Initialize Keras OCR pipeline
    pipeline = keras_ocr.pipeline.Pipeline()

    # Iterate over the files in the input folder
    for filename in os.listdir(input_folder_path):
        # Check if the file is an image (you may want to add more file extensions)
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            # Read the image in color
            img_path = os.path.join(input_folder_path, filename)
            img = cv2.imread(img_path)

            # Perform text detection
            prediction_groups = pipeline.recognize([img])

            # Create a mask for text regions
            mask = np.zeros(img.shape[:2], dtype="uint8")
            for idx, box in enumerate(prediction_groups[0]):
                if idx >= max_text_regions:
                    break  # Limit the number of text regions processed
                x0, y0 = box[1][0]
                x1, y1 = box[1][1]
                x2, y2 = box[1][2]
                x3, y3 = box[1][3]

                x_mid0, y_mid0 = midpoint(x1-100, y1-100, x2+100, y2+100)
                x_mid1, y_mid1 = midpoint(x0+100, y0+100, x3-100, y3-100)

                thickness = int(math.sqrt((x2 - x1)**2 + (y2 - y1)**2) + 20)

                cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mid1), 255, thickness)

            # Inpaint text regions
            inpainted_img = cv2.inpaint(img, mask, 10, cv2.INPAINT_NS)

            # Save the inpainted image to the output folder
            output_img_path = os.path.join(output_folder_path, filename)
            cv2.imwrite(output_img_path, inpainted_img)
            print(f"Processed image saved: {output_img_path}")




Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 7s 7s/step
Processed image saved: output_folder1/15.jpg


1/1 [==============================] - 4s 4s/step
Processed image saved: output_folder1/48.jpg
1/1 [==============================] - 6s 6s/step
Processed image saved: output_folder1/55.jpg
1/1 [==============================] - 5s 5s/step
Processed image saved: output_folder1/51.jpg
1/1 [==============================] - 4s 4s/step
Processed image saved: output_folder1/2.jpg
1/1 [==============================] - 4s 4s/step
Processed image saved: output_folder1/21.jpg
1/1 [==============================] - 3s 3s/step
Processed image saved: output_folder1/9.jpg
1/1 [==============================] - 7s 7s/step
Processed image saved: output_folder1/52.jpg
1/1 [==============================] - 5s 5s/step
Processed image saved: output_folder1/44.jpg


# using

In [ ]:
# Example usage:
input_folder_path = 'ids'
output_folder_path = 'output_folder1'
inpaint_text(input_folder_path, output_folder_path)

# SIFT

In [2]:
import cv2
import numpy as np
import os

def remove_logo_from_images(input_folder, output_folder ):
    # Initialize feature detectors
    sift = cv2.SIFT_create()
    orb = cv2.ORB_create()
    logo_path ='logo.jpg'
    # Read the logo image
    logo_image = cv2.imread(logo_path)
    gray_logo = cv2.cvtColor(logo_image, cv2.COLOR_BGR2GRAY)
    gray_logo = cv2.GaussianBlur(gray_logo, (5, 5), 0)
    gray_logo = cv2.equalizeHist(gray_logo)

    # Compute descriptors for the logo
    kp_logo, des_logo = sift.detectAndCompute(gray_logo, None)
    kp_logo_orb, des_logo_orb = orb.detectAndCompute(gray_logo, None)

    # Create FLANN matcher for SIFT
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Create BFMatcher for ORB
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
            gray_image = cv2.equalizeHist(gray_image)

            # Compute descriptors for the image
            kp_image, des_image = sift.detectAndCompute(gray_image, None)
            kp_image_orb, des_image_orb = orb.detectAndCompute(gray_image, None)

            # Match SIFT descriptors
            matches = flann.knnMatch(des_logo, des_image, k=2)
            good_matches = [m for m, n in matches if m.distance < 0.6 * n.distance]

            # Match ORB descriptors
            matches_orb = bf.match(des_logo_orb, des_image_orb)
            good_matches_orb = sorted(matches_orb, key=lambda x: x.distance)

            # Combine good matches from both descriptors
            good_matches_combined = good_matches + good_matches_orb[:min(len(good_matches_orb), len(good_matches))]

            if len(good_matches_combined) > 10:
                # Extract locations of good matches
                src_pts = np.float32([kp_logo[m.queryIdx].pt for m in good_matches_combined if m.queryIdx < len(kp_logo)]).reshape(-1, 1, 2)
                dst_pts = np.float32([kp_image[m.trainIdx].pt for m in good_matches_combined if m.trainIdx < len(kp_image)]).reshape(-1, 1, 2)

                # Compute homography
                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 3.0)
                h, w, _ = logo_image.shape

                # Get coordinates of the logo in the target image
                pts = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
                dst = cv2.perspectiveTransform(pts, M)

                # Mask the logo area
                logo_mask = np.zeros_like(gray_image)
                cv2.fillPoly(logo_mask, [np.int32(dst)], 255)

                # Inpaint the logo area
                result = cv2.inpaint(image, logo_mask, 3, cv2.INPAINT_TELEA)

                # Save the result in the output folder
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, result)
                print(f"Processed {filename} and saved to {output_path}")

    print("Processing completed for all images.")




Processed 15.jpg and saved to output_folder/15.jpg
Processed 48.jpg and saved to output_folder/48.jpg
Processed 55.jpg and saved to output_folder/55.jpg
Processed 51.jpg and saved to output_folder/51.jpg
Processed 2.jpg and saved to output_folder/2.jpg
Processed 21.jpg and saved to output_folder/21.jpg
Processed 9.jpg and saved to output_folder/9.jpg
Processed 52.jpg and saved to output_folder/52.jpg
Processed 44.jpg and saved to output_folder/44.jpg
Processing completed for all images.


# using

In [ ]:
# Example usage
input_folder = 'ids'
output_folder = 'output_folder'
remove_logo_from_images(input_folder, output_folder)